### Basic Imports

In [4]:
# Import everything and setup things
import sys
import os
file_folder = os.getcwd()
if "testing_folder" in file_folder:
    file_folder = os.path.dirname(file_folder)
    sys.path.append(file_folder)
    os.chdir(file_folder)

import datetime
import math
import time
import json
from youtube_transcript_api import YouTubeTranscriptApi
import tiktoken
import faiss
import numpy as np
import asyncio
import ipywidgets
from IPython.display import display, HTML
import imp


keys=json.load(open("working_folder/keys.json"))

# Setup anthropic
import anthropic
anthropic_client = anthropic.Anthropic(api_key=keys["anthropic"])

# Setup openai
from openai import OpenAI
os.environ["OPENAI_API_KEY"]=keys["openai"]
openai_client = OpenAI(api_key=keys["openai"])

enc=tiktoken.get_encoding("cl100k_base")
input_cost=0
output_cost=0

### Import custom library or Reload for development

In [5]:
# import and reload custom functions 
import AiFunctions as af
imp.reload(af)

<module 'AiFunctions' from 'd:\\DestinyFolder\\DestinyAIBot\\AiFunctions.py'>

## Designate Video ID for all subsequent processes https://vyneer.me/vods/

In [6]:
video_url="https://youtu.be/savIUlLu0MY"
# Set video id
#video_id="FC3giY7YDAQ"
#video_id="hkOI46Ai4g8"
video_id="YZuSc-EnI3o"

try:
    if video_url!="":
        video_id=video_url.split("/")[-1]
except:
    pass


### Load or Get transcript

In [7]:
# Get transcript and nearest times
transcript, nearest_times=af.load_or_get_transcript_info(video_id)

Getting transcript from YouTube


### Load or Make text chunks and vector_db for transcript search

In [8]:
vector_db, text_chunks_dict = await af.load_or_get_vectordb_and_chunks(openai_client, transcript, nearest_times, video_id)

Generating text chunks and vector db
Number of chunks:  417
Finished generating embeddings


### Search Transcript

In [9]:
#load from text_chunks_dict
user_query=input("Enter your query: ")
print("Searching for:", user_query)

D, I=af.search_vector_db(openai_client, vector_db, user_query,5)

thtml=af.convert_to_html(nearest_times, video_id, text_chunks_dict[str(I[0][0])]["text"],text_chunks_dict[str(I[0][0])]["start"],I[0][0])
display(HTML(thtml))

Searching for: test


### Create summarized segments

In [10]:
model_responses, run_input_cost, run_output_cost=await af.load_or_make_summarized_segments(anthropic_client, transcript, nearest_times, video_id, segments=1000)
input_cost+=run_input_cost
output_cost+=run_output_cost

Approximate cost:  0.5603069999999999   Number of segments:  42
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 Error: 27 Error: 31 Error: 28 Error: 30 32 33 34 35 36 37 38 Error: 33 Error: 34 39 Error: 36 40 41 Error: 38 Error: 39 Retrying: 27 27 Retrying: 31 31 Retrying: 28 28 Retrying: 30 30 Error: 41 Error: 27 Error: 31 Error: 30 Retrying: 33 33 Retrying: 34 34 Retrying: 36 36 Retrying: 38 38 Error: 34 Retrying: 39 39 Error: 36 Retrying: 41 41 Error: 38 Error: 41 Retrying: 27 27 Retrying: 31 31 Retrying: 30 30 Retrying: 34 34 Retrying: 36 36 Retrying: 38 38 Retrying: 41 41 Total Cost: 0.501012
Model responses saved to file


### Make meta summary

In [12]:
meta_summary=af.load_or_make_meta_summary(anthropic_client, model_responses, video_id)

Generating meta summary
Expected cost:  0.041445
Here is a summary of the main topics and key details based on the collection of summaries:

Main Topics:

1. Discussions on the Israeli-Palestinian conflict, analyzing whether Israel's actions constitute genocide based on legal definitions and available evidence. Destiny seems skeptical that it rises to the level of genocide currently.

2. Analysis of college costs, earnings differentials, and the economic value of degrees versus alternative career paths like skilled trades. Destiny presents a nuanced view that higher education's payoff depends significantly on individual circumstances.

3. Responding to personal allegations and controversies around inappropriate interactions with underage individuals and mishandling consent situations. Destiny denies intentional misconduct but acknowledges missed cues.  

4. Concerns about foreign government disinformation campaigns using social media to amplify social/political division and radicalizat

# Dev
### Chat processing

In [13]:
from chat_downloader import ChatDownloader

chat = ChatDownloader().get_chat(video_url)       # create a generator
c_count=0
all_chat_messages=[]
all_chat_messages_str=""
for message in chat:                        # iterate over messages
    #chat.print_formatted(message)   
    chat_message_formatted="User: "+message["author"]["name"] +", "+message["time_text"]+", Message: "+message["message"]
    all_chat_messages.append(chat_message_formatted)
    all_chat_messages_str+=chat_message_formatted+"\n"
    if c_count%100==0:
        print(chat_message_formatted) 
    c_count+=1
print(c_count)

# save chat string and messages
with open("working_folder/"+video_id+"/chat.txt","w", encoding='utf-8') as f:
    f.write(all_chat_messages_str)
json.dump(all_chat_messages,open("working_folder/"+video_id+"/chat.json","w"))
print("saved chat")

User: Aimsforgroin, 0:21, Message: W
User: Beezo, 2:56, Message: :shelterin:mr bonelli*
User: geeese jeeeze, 6:07, Message: bald:_destinyPepeW::_destinyPepeW:
User: Caged, 10:26, Message: can somebody define hoflation please
User: geeese jeeeze, 14:31, Message: its like Itachi's Amaterasu
User: Prof Crow, 19:21, Message: we love greece and romania, fuck turkey doe
User: Stigala, 22:55, Message: nah wouldnt call it a genocide potential/failed annexation of ukraine
User: Wilson, 26:05, Message: Is it genocide if you don’t try to kill the entire population, just the population of them in the region you’re trying to take over? 
User: Raiders Cat⚔, 31:44, Message: @Naga cute pfp:shelterin::_destinyDggL:
User: Dogwater, 36:00, Message: 2 GRUMS 
User: Will sr, 39:18, Message: So is avatar not a genocide because the intent wasn’t to kill the blue avatar just to take pandora?
User: Grum, 41:49, Message: @Wilson thx I know
User: alpha, 45:57, Message: :gorilla::gorilla:
User: Wilson, 52:06, Mess

In [14]:
# load all chat messages
all_chat_messages=json.load(open("working_folder/"+video_id+"/chat.json"))

In [31]:
transcript_segment

{'bot': 'Here is a synopsis of the topics and sentiments expressed in the transcript:\n\nSuicide and Men\'s Mental Health:\n- The speaker (Destiny) agrees with Dr. K\'s viewpoint that many suicidal men are not clinically mentally ill, but rather feel their lives are not worth living due to their circumstances/outlook.\n- He expresses that this perspective comes from empathy and trying to understand their mindset, rather than immediately dismissing it as wrong.\n- He argues that much self-help content online just validates people\'s existing harmful behaviors rather than challenging them to improve.\n\nJordan Peterson:\n- Destiny believes Jordan Peterson was respected prior to his controversial takes around the C-16 bill.\n- He sees Peterson\'s clinical psychology practice as respectable, but dismisses his more recent cultural/political commentary as "garbage."\n\nOverall, the main sentiments are:\n1) Empathizing with perspectives you disagree with, rather than just dismissing them.\n2)

In [46]:
# get chat segment for summary segment
transcript_segment=model_responses[0]

start_time=transcript_segment["start_second"]
end_time=transcript_segment["end_second"]



chats_in_segment=[]
chats_txt=""
chat_before_end=True
i=0
while chat_before_end:
    try:
        # Get chat time
        chat_msg_time=all_chat_messages[i].split(",")[1].split(" ")[-1]
        if chat_msg_time.count(":")==1:
            chat_time=datetime.datetime.strptime(chat_msg_time, '%M:%S')
            chat_time=chat_time.minute*60+chat_time.second
        else:
            chat_time=datetime.datetime.strptime(chat_msg_time, '%H:%M:%S')
            chat_time=chat_time.hour*3600+chat_time.minute*60+chat_time.second
        
        if chat_time>end_time:
            chat_before_end=False
        elif chat_time>start_time:
            #print(all_chat_messages[i])
            chats_in_segment.append(all_chat_messages[i])
            chats_txt+=all_chat_messages[i]+"\n"
    except:
        pass
    i+=1
            

# # Get all chat messages in segment
# for i in range(len(all_chat_messages)):
#     try:
#         chat_msg_time=all_chat_messages[i].split(",")[1].split(" ")[-1]
#         if chat_msg_time.count(":")==1:
#             chat_time=datetime.datetime.strptime(chat_msg_time, '%M:%S')
#             chat_time=chat_time.minute*60+chat_time.second
#         else:
#             chat_time=datetime.datetime.strptime(chat_msg_time, '%H:%M:%S')
#             chat_time=chat_time.hour*3600+chat_time.minute*60+chat_time.second
#     except:
#         pass

#while 

In [57]:
# Chat summary
bot_response=""
chat_summary_prompt="You are meant to summarize serious messages in chat and get a general sentiment of the chat as well as pull out the repeated serious sentiments or serious questions or on topic messages. The user will give you all the chat messages with username\nMessage.\n\nYou will also be given the summary of the transcript of the livestream that the chat is facing which you can use to evalute who is on topic and better sense sentiment."

user_prompt="Here is the transcript summary: {transcript_summary}\n\nHere is the chat: {chat}"
f_user_prompt=user_prompt.format(transcript_summary=transcript_segment["bot"], chat=chats_txt)

print(len(enc.encode(f_user_prompt)))
proceed_str=input("Proceed with chat summary? (y/n)")
if proceed_str=="y":        
    with anthropic_client.messages.stream(
            max_tokens=4096,
            system=chat_summary_prompt,
            messages=[{"role":"user", "content": user_prompt.format(transcript_summary=transcript_segment["bot"], chat=chats_txt)}],
            model="claude-3-sonnet-20240229",
            #model="claude-3-haiku-20240307"
        ) as stream:
            for text in stream.text_stream:
                bot_response+=text
                print(text, end="", flush=True)
else:
    print("Process cancelled")

8390
Here are the main serious sentiments, questions and on-topic messages I gathered from the chat:

Suicide and Men's Mental Health:
- Chat empathized with Destiny's perspective of trying to understand the mindset of suicidal men who feel their lives are not worth living due to circumstances, rather than just dismissing it. There was agreement that much online self-help content just validates harmful behaviors rather than challenging people to improve.

Jordan Peterson:
- There was respect expressed for Jordan Peterson's past clinical psychology work, but criticism of his more recent political/cultural commentary as "garbage."
- Some felt Peterson became respected by offering basic self-improvement advice like "clean your room", but then went off the rails.

Empathy vs Validation:
- Chat agreed with the importance of empathizing with perspectives you disagree with, rather than just dismissively validating harmful behaviors/mindsets. 
- However, some argued that for people in really d

In [58]:
# Where
print(len(enc.encode(f_user_prompt)))
proceed_str=input("Proceed with chat summary? (y/n)")
chat_query_prompt=" For each topic you are meant to pull out each individual comment that was mentioned to be related to that topic. You have to be thourough."
chat_query_response=""
if proceed_str=="y":        
    with anthropic_client.messages.stream(
            max_tokens=4096,
            system=chat_summary_prompt,
            messages=[{"role":"user", "content": user_prompt.format(transcript_summary=transcript_segment["bot"], chat=chats_txt)}, {"role":"assistant", "content":bot_response},{"role":"user", "content": "Can you say which comments you are reffering to now? Mention what claim they support and their full message including username and time."}],
            #model="claude-3-sonnet-20240229",
            model="claude-3-haiku-20240307"
        ) as stream:
            for text in stream.text_stream:
                chat_query_response+=text
                print(text, end="", flush=True)
else:
    print("Process cancelled")

8390
Sure, here are the specific comments I'm referring to:

Suicide and Men's Mental Health:
- "Chat empathized with Destiny's perspective of trying to understand the mindset of suicidal men who feel their lives are not worth living due to circumstances, rather than just dismissing it." This is supported by comments like:
"I'm not suicidal:_destinyDaFeels:" from Raiders Cat⚔ at 4:44
":_destinyDaFeels::_destinyDaFeels::_destinyDaFeels:" from Anirudh Prakash at 4:45

- "There was agreement that much online self-help content just validates harmful behaviors rather than challenging people to improve." This is supported by comments like: 
"It's the responsibility inherent in determinism that you need to take constant action" from Fung Dark at 5:37
"cbt teaches u to change ur mind" from geeese jeeeze at 5:38

Jordan Peterson:
- "There was respect expressed for Jordan Peterson's past clinical psychology work, but criticism of his more recent political/cultural commentary as 'garbage.'" This 